In [1]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
bounds = ee.Geometry.BBox(-20,-40,60,60)

In [17]:
ndvicollection = (ee.ImageCollection('MODIS/061/MOD13A2')
    .filterBounds(bounds)
    .select(['NDVI', 'DetailedQA'])
    #.select('sm_surface')
    )

In [5]:
def bitwiseExtract(value, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return value.rightShift(fromBit).bitwiseAnd(mask)

In [19]:
def mask_ndvi(image):
    qa_image = image.select('DetailedQA')
    ndvi_image = image.select('NDVI')
    mask = bitwiseExtract(qa_image, 0, 1).eq(0)
    masked_ndvi = ndvi_image.updateMask(mask)
    return masked_ndvi

In [68]:
masked_ndvi = ndvicollection.map(mask_ndvi)

In [82]:
start = ee.Date('2015-04-01')
end = ee.Date('2023-01-01')
n_months = end.difference(start, 'month').subtract(1)
yearmonths = ee.List.sequence(0, n_months).map(lambda n : start.advance(n, 'month'))

In [76]:
def makeYearMonthlyComposite(date):
  date = ee.Date(date)
  
  composite = (masked_ndvi
        .filterDate(date, date.advance(1, 'month'))
        .mean()
        .set("system:index", date.format("YYYY-MM")))
  
  return composite

In [77]:
yearmonthly_ndvi = ee.ImageCollection(yearmonths.map(makeYearMonthlyComposite))

In [109]:
def makeMonthlyComposite(month):
  
  filter = ee.Filter.calendarRange(month, month, 'month')
  
  composite = (ndvicollection.select('NDVI')
            .filter(filter)
            .mean())
  
  return composite

In [110]:
months = ee.List([*range(1,13)])

monthly_ndvi = ee.ImageCollection(months.map(makeMonthlyComposite)).toBands()

In [113]:
task = ee.batch.Export.image.toDrive(
    image=monthly_ndvi,
    description=f'monthly_ndvi_nomask',
    folder = 'ndvi',
    region = bounds,
    dimensions = "1920x2400",
    #scale = 5000,
    #crs = 'EPSG:25832'
)
task.start()

In [114]:
task.status()

{'state': 'RUNNING',
 'description': 'monthly_ndvi_nomask',
 'creation_timestamp_ms': 1679045820572,
 'update_timestamp_ms': 1679046175653,
 'start_timestamp_ms': 1679045875072,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'WHMGFV2SJONCFDK5BXEIJBY6',
 'name': 'projects/earthengine-legacy/operations/WHMGFV2SJONCFDK5BXEIJBY6'}